## 3.1 Tensor

In [2]:
import torch as t

### 3.1.1 基础操作

从接口角度讲，对 tensor 的操作可以分为两类
- `torch.function`, 如 `torch.save`
- `tensor.function`, 如 `tensor.view`

从存储的角度讲，对 tensor 的操作可以分为两类
- 不会修改自身的数据 `a.add(b)`
- 会修改自身的数据 `a.add_(b)` inplace 方式

**创建 Tensor**

常见的新建 tensor 方法

| 函数 | 功能 |
| :------| ------: |
| Tesor(*sizes)  | 基础构造函数 |
| ones(*sizes) | 全 1 Tensor |
| zeros(*sizes) | 全 0 Tensor |
| eye(*sizes) | 对角线为 1, 其他为 0 |
| arange(s, e, steps) | 从 s 到 e, 步长为 step |
| linspace(s, e, steps) | 从 s 到 e, 均匀切分成 steps 份 |
| rand/randn(*sizes) | 均匀 / 标准正态分布 |
| normal(mean, std) / uniform(form, to) | 正态分布 / 均匀分布 |
| randperm(m) | 随机排列 |


In [3]:
# 指定 tensor 的形状
a = t.Tensor(2, 3)
a  # 数值取决于内存空间的状态

tensor([[-6.1866e+13,  3.0735e-41,  4.9656e+28],
        [ 4.5439e+30,  5.7453e-44,  0.0000e+00]])

In [4]:
# 用 list 的数据创建 tensor
b = t.Tensor([[1,2,3], [4,5,6]])
b

tensor([[1., 2., 3.],
        [4., 5., 6.]])

In [5]:
b.tolist()  # 把 tensor 转为 list

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

`tensor.size()` 返回 `torch.Size` 对象，它是 tuple 的子类

In [6]:
b_size = b.size()
b_size

torch.Size([2, 3])

In [7]:
b.numel()  # b 中元素总个数， 2*3 等价于 b.nelement()

6

In [9]:
# 创建一个和 b 形状一样的 tensor
c = t.Tensor(b_size)
# 创建一个元素为 2 和 3 的 tensor
d = t.Tensor((2, 3))
print(c)
print(d)

tensor([[-1.5843e+09,  4.5566e-41, -1.8455e+13],
        [ 3.0735e-41,  4.4842e-44,  0.0000e+00]])
tensor([2., 3.])


还可以用 `tensor.shape` 直接查看 tensor 的形状

`tensor.shape` 等价于 `tensor.size()`

In [10]:
c.shape

torch.Size([2, 3])

注意：

`t.Tensor(*sizes)`创建 tensor 时，系统不会马上分配空间，

只会计算剩余的内存是否够用，使用到 tensor 时才会分配，

而其他操作都是在创建完成 tensor 后马上进行空间分配。

In [14]:
t.ones(2, 3)

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [16]:
t.zeros(2, 3)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [18]:
t.arange(1, 6, 2)

tensor([1, 3, 5])

In [19]:
t.linspace(1, 10, 3)

tensor([ 1.0000,  5.5000, 10.0000])

In [30]:
t.randn(2, 3)

tensor([[ 1.1608,  1.6276, -0.6581],
        [-0.5305,  0.2952,  0.0953]])

In [31]:
t.randperm(5)  # 长度为 5 的随机排列

tensor([2, 4, 0, 3, 1])

In [32]:
t.eye(2, 3)  # 对角线为 1, 不要求行列数一致

tensor([[1., 0., 0.],
        [0., 1., 0.]])

**常用 Tensor 操作**

通过`tensor.view`方法可以调整 tensor 的形状，

但必须保证调整前后元素个数一致。

view 不会修改自身的数据，返回的新的 tensor 与源 tensor 共享内存

实际应用中经常需要添加或减少某一维度，用`squeeze`和`unsqueeze`

In [33]:
a = t.arange(0, 6)
print(a)
a.view(2, 3)

tensor([0, 1, 2, 3, 4, 5])


tensor([[0, 1, 2],
        [3, 4, 5]])

In [34]:
b = a.view(-1, 3)  # 当某一维为 -1 的时候，会自动计算它的大小
b

tensor([[0, 1, 2],
        [3, 4, 5]])

In [35]:
b.unsqueeze(1)  # 在第 1 维(下标从0开始)上增加 “1”

tensor([[[0, 1, 2]],

        [[3, 4, 5]]])

In [36]:
b.unsqueeze(-2)

tensor([[[0, 1, 2]],

        [[3, 4, 5]]])

In [38]:
c = b.view(1, 1, 1, 2, 3)
c.squeeze(0)  # 压缩第 0 维的 “1”

tensor([[[[0, 1, 2],
          [3, 4, 5]]]])

In [39]:
c.squeeze()  # 把所有维度为 “1” 的压缩

tensor([[0, 1, 2],
        [3, 4, 5]])

In [40]:
a[1] = 100
b  # a 和 b 共享内存，修改了a, b也变了

tensor([[  0, 100,   2],
        [  3,   4,   5]])

`resize`可以用来调整 size，与 view 不同，它可以修改 tensor 尺寸

如果新尺寸超过了原尺寸，则会分配新的内存空间，如果小于，则之前的数据会保留

In [41]:
b.resize_(1, 3)
b

tensor([[  0, 100,   2]])

In [42]:
b.resize_(3, 3)  # 级的数据依旧保存着，多出的数据会分配新的空间
b

tensor([[              0,             100,               2],
        [              3,               4,               5],
        [            143, 139662924089920,              80]])

**索引操作**

如无特殊说明，索引出来的结果与原 tensor 共享内存

In [43]:
a = t.randn(3, 4)
a

tensor([[ 6.6869e-02, -2.0206e-01, -9.6794e-04, -1.1772e+00],
        [ 1.1646e+00, -1.3940e-01, -4.5974e-01,  3.1210e-01],
        [-3.8002e-01,  1.4402e+00, -1.2241e+00,  1.4215e-01]])

In [44]:
a[0]  # 第 0 行

tensor([ 6.6869e-02, -2.0206e-01, -9.6794e-04, -1.1772e+00])

In [45]:
a[:, 0]  # 第 0 列

tensor([ 0.0669,  1.1646, -0.3800])

In [46]:
a[0][2]  # 第 0 行  第 2 个元素 等价于 a[0, 2]

tensor(-0.0010)

In [47]:
a[0, -1]  # 第 0 行最后一个元素

tensor(-1.1772)

In [48]:
a[:2]  # 前两行

tensor([[ 6.6869e-02, -2.0206e-01, -9.6794e-04, -1.1772e+00],
        [ 1.1646e+00, -1.3940e-01, -4.5974e-01,  3.1210e-01]])

In [49]:
a[:2, 0:2]  # 前两行，第 0,1 列

tensor([[ 0.0669, -0.2021],
        [ 1.1646, -0.1394]])

In [50]:
print(a[0:1, :2])  # 第 0 行，前两列
print(a[0, :2])  # 注意两者的区别： 形状不同

tensor([[ 0.0669, -0.2021]])
tensor([ 0.0669, -0.2021])


In [51]:
a > 1  # 返回一个 ByteTensor

tensor([[0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0]], dtype=torch.uint8)

In [52]:
a[a>1]  # 等价于 a.masked_select(a>1)
# 选择结果与原 tensor 不共享内存空间

tensor([1.1646, 1.4402])

In [53]:
a[t.LongTensor([0, 1])]  # 第 0 行和第 1 列

tensor([[ 6.6869e-02, -2.0206e-01, -9.6794e-04, -1.1772e+00],
        [ 1.1646e+00, -1.3940e-01, -4.5974e-01,  3.1210e-01]])

其他常用的选择函数

| 函数 | 功能 |
| :------| ------: |
| index_select(input, dim, index)  | 在指定维度 dim 上选取，例如选取某些行、某些列 |
| masked_select(input, mask) | 使用 ByteTensor 进行选取 |
| non_zero(input) | 非 0 远古三的下标 |
| gather(input, dim, index) | 根据 index, 在 dim 维度上选取数据，输出的 size 与 index 一样 |

gather 是一个比较复杂的操作，对一个二维 tensor，输出的每个元素如下：

In [ ]:
out[i][j] = input_[index[i][j]][j]  # dim=0
out[i][j] = input_[i][index[i][j]]  # dim=0

In [54]:
a = t.arange(0, 16).view(4, 4)
a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])

In [55]:
# 选取对角线的元素
index = t.LongTensor([[0, 1, 2, 3]])
a.gather(0, index)

tensor([[ 0,  5, 10, 15]])

In [57]:
# 选取反对角线上的元素
index = t.LongTensor([[3, 2, 1, 0]]).t()
a.gather(1, index)

tensor([[ 3],
        [ 6],
        [ 9],
        [12]])

In [58]:
# 选取反对角线上的元素, 注意与上边的不同
index = t.LongTensor([[3, 2, 1, 0]])
a.gather(0, index)

tensor([[12,  9,  6,  3]])

In [59]:
# 选取两个对角线上的元素
index = t.LongTensor([[0, 1, 2, 3], [3, 2, 1, 0]]).t()
b = a.gather(1, index)
b

tensor([[ 0,  3],
        [ 5,  6],
        [10,  9],
        [15, 12]])